In [21]:
import pandas as pd
import random
import string

# ✅ 변경하지 않을 문자 (숫자 + 특수문자)
SPECIAL_CHARACTERS = set(string.punctuation + string.digits)  # 특수문자 + 숫자 ('0'-'9' 포함)

def replace_with_random_output_chars(input_text: str, output_chars: list) -> str:
    """
    input_text에서 랜덤한 비율(0~30%)만큼 문자를 output_chars에서 랜덤하게 선택하여 치환 (공백, 숫자, 특수문자 제외)
    """
    input_chars = list(input_text)

    replace_ratio = random.uniform(0, 0.3)  # ✅ 0~30% 사이 랜덤 비율 설정
    num_replacements = int(len(input_chars) * replace_ratio)  # 변형할 문자 개수 결정

    # ✅ 변경할 위치 선택 (공백, 숫자, 특수문자는 제외)
    valid_positions = [i for i in range(len(input_chars)) if input_chars[i] not in SPECIAL_CHARACTERS and input_chars[i] != " "]
    num_replacements = min(num_replacements, len(valid_positions))  # 가능한 위치 수보다 크면 조절
    replace_positions = random.sample(valid_positions, num_replacements)  # 랜덤 위치 선택

    for pos in replace_positions:
        # ✅ 변형된 글자가 특수문자 또는 숫자가 되지 않도록 필터링
        new_char = random.choice(output_chars)
        while new_char in SPECIAL_CHARACTERS:  # 숫자 또는 특수문자가 나오지 않도록 반복
            new_char = random.choice(output_chars)

        input_chars[pos] = new_char  # output 전체에서 랜덤한 글자 삽입

    return "".join(input_chars)

# 데이터 불러오기
df = pd.read_csv("../database/train/train.csv")  # CSV 파일 읽기

# Output 데이터 전체에서 음절 리스트 만들기 (공백, 숫자, 특수문자 제외)
all_output_chars = list("".join(df["output"].astype(str).values).replace(" ", ""))
all_output_chars = [char for char in all_output_chars if char not in SPECIAL_CHARACTERS]  # ✅ 숫자 및 특수문자 제거

# 1. 기존 데이터 유지
df_augmented = df.copy()

# 2. OUTPUT-OUTPUT 쌍 추가
df_output_output = df[["output"]].copy()
df_output_output.columns = ["input"]  # output을 input으로 사용
df_output_output["output"] = df_output_output["input"]

# 3. INPUT을 변형한 데이터 추가 (각 문장 당 10개씩)
new_rows = []
for _, row in df.iterrows():
    for _ in range(20):  # ✅ 각 문장당 10개 변형
        replaced_input = replace_with_random_output_chars(row["output"], all_output_chars)  # 0~30% 랜덤 비율로 치환 (공백, 숫자, 특수문자 제외)
        new_rows.append({"ID": row["ID"], "input": replaced_input, "output": row["output"]})  # 변형된 문장 추가

# 새로운 데이터 합치기
df_augmented = pd.concat([df_output_output, pd.DataFrame(new_rows)], ignore_index=True)

# 증강된 데이터 저장
df_augmented.to_csv("../database/train/train_augmented.csv", index=False)
print(f"✅ 데이터 증강 완료! 저장 경로: ../database/train/train_augmented.csv")


✅ 데이터 증강 완료! 저장 경로: ../database/train/train_augmented.csv


In [22]:
# 저장된 파일 불러와서 검증 후 공백 제거
df_saved = pd.read_csv("../database/train/train_augmented.csv")

# ✅ ID 열 삭제 (ID 열이 존재하는 경우)
if "ID" in df_saved.columns:
    df_saved = df_saved.drop(columns=["ID"])

# 입력과 출력 길이 검증 후 공백 제거
df_saved.loc[df_saved["input"].str.len() != df_saved["output"].str.len(), "input"] = df_saved["input"].str.strip()
df_saved.loc[df_saved["input"].str.len() != df_saved["output"].str.len(), "output"] = df_saved["output"].str.strip()

# 입력과 출력 길이 일치 여부 출력
length_match_counts = (df_saved["input"].str.len() == df_saved["output"].str.len()).value_counts()
print("저장된 파일 - 입력과 출력 길이 일치 여부:")
print(length_match_counts)

# ✅ 수정된 데이터 저장
df_saved.to_csv("../database/train/train_augmented.csv", index=False)


저장된 파일 - 입력과 출력 길이 일치 여부:
True    236523
dtype: int64
